In [1]:
import pandas as pd

In [2]:
orders = pd.read_excel('orders-test.xlsx')
graphic = pd.read_excel('graphic-test.xlsx')

In [3]:
orders

,КССС мат,Пункт отгрузки,"осталось произвести, тн"
0,1779443,ВНП,0.350
1,1779443,ВНП,0.525
2,1779443,ВНП,1.400
3,1779443,ВНП,2.100
4,1779443,ВНП,0.350
5,1779443,ВНП,1.400
6,1779443,ВНП,0.175
7,1779443,ВНП,0.350
8,1779443,ВНП,0.700


In [4]:
graphic

,Пункт отгрузки,Код КССС,Дата_производства,"Объем_фасовки, т"
0,ВНП,1779443,2022-06-11,4
1,ВНП,1779443,2022-06-12,3


In [5]:
#graphic.drop(graphic[(graphic['Пункт отгрузки']=='ВНП') & (graphic['Код КССС']==1779443) & 
                           # (graphic['Дата_производства']=='2022-06-11')].index, inplace=True)

In [6]:
#graphic.loc[(graphic['Пункт отгрузки']=='ВНП') & (graphic['Код КССС']==1779443) & 
                          #  (graphic['Дата_производства']=='2022-06-11'), 'Объем_фасовки, т'] = 4

In [7]:
#graphic[(graphic['Пункт отгрузки']=='ПНОС') & (graphic['Код КССС']==207465)]

In [8]:
#len_of_dates = len(graphic.loc[(graphic['Пункт отгрузки']=='ПНОС') & (graphic['Код КССС']==207465)])

In [9]:
#graphic.loc[(graphic['Пункт отгрузки']=='ПНОС') & (graphic['Код КССС']==207465) & (graphic['Дата_производства']=='2022-06-10'), 'Объем_фасовки, т']

In [10]:
#graphic.loc[(graphic['Пункт отгрузки']=='ПНОС') & (graphic['Код КССС']==207465) & 
#                        (graphic['Дата_производства']=='2022-06-10'), 'Объем_фасовки, т'] 

In [11]:
#graphic.loc[(graphic['Пункт отгрузки']=='ПНОС') & (graphic['Код КССС']==207465), 'Дата_производства'].iloc[0]

In [12]:
def data_shipment(city, code, required_to_make):
    print('НАЧАЛО')
    ## проверяем есть ли вообще в списке graphic
    if graphic[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code)].empty:
        return '-'
    ## если в таблице orders не требуется, то мы ставим черточку
    elif required_to_make == 0:
        return '-'
    ## иначе если есть и объема в графике достаточно, то мы    
    elif graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code), 'Объем_фасовки, т'].iloc[0] >= required_to_make:   
        print('Объем больше')
        
        ## вычитаем из объема
        ## добавить определенное дату, так как будет удалять все значения где код и пункт совпадаем
        data_of_making = graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code) & 
                                    (graphic['Дата_производства']==graphic.loc[(graphic['Пункт отгрузки']==city) 
                                    & (graphic['Код КССС']==code), 'Дата_производства'].iloc[0]),'Дата_производства'].iloc[0]
        graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code) & (graphic['Дата_производства']==data_of_making),
                    'Объем_фасовки, т'] -= required_to_make
        # и возвращаем дату
        data = graphic[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code)]['Дата_производства'].dt.date.iloc[0]
        print('graphic1: ', graphic)
        return data
    ## иначе если недостаточно объема чем требуется ...
    else:
        print('объем меньше')
        if len(graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code)]) > 1:
            ## вводим переменную длины, чтобы не выйти за пределы
            len_of_dates = len(graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code)])
            ## вводим счетчик для того чтобы присвоить правильную дату
            i = 0
            ## создаем накопительную переменную 
            storage_var = graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code), 'Объем_фасовки, т'].iloc[0]


            ## пока объем фасовки меньше, чем требуется
            while storage_var<required_to_make and len_of_dates>1:
                print('в цикле')
                print('len_of_dates=', len_of_dates)
                len_of_dates-=1
                ## переходим к следующей
                i+=1
                ## и прибавляем к накопительной переменной
                storage_var+= graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code), 'Объем_фасовки, т'].iloc[i]
                print('прибавили к накопительной следующий объем: ', storage_var)
                 ## и обнуляем в графике объем фасовки
                #graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code), 'Объем_фасовки, т'].iloc[i]
                
                print('storage fresh, ', storage_var)
                print('required fresh, ' , required_to_make) 
                
            #если требуемый объем не накопился то возвращает -
            if storage_var<required_to_make:
                print('не накопился')
                return '-'
            # иначе
            else:
                ## предыдущая дата и текущая
                data_of_making_pred = graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code) & 
                                             (graphic['Дата_производства']==graphic.loc[(graphic['Пункт отгрузки']==city) 
                                            & (graphic['Код КССС']==code), 'Дата_производства'].iloc[i-1]),
                                             'Дата_производства'].iloc[0]
                data_of_making = graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code) & 
                                             (graphic['Дата_производства']==graphic.loc[(graphic['Пункт отгрузки']==city) 
                                            & (graphic['Код КССС']==code), 'Дата_производства'].iloc[i]),
                                             'Дата_производства'].iloc[0]
                


                print('обнуляем предыдущую дату')
                graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code) & 
                            (graphic['Дата_производства']==data_of_making_pred), 'Объем_фасовки, т'] = 0
                print('после обнуления:, ',graphic)

            print('data making', data_of_making)
        
            ## вычитаем
            graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code) & 
                        (graphic['Дата_производства']==data_of_making), 'Объем_фасовки, т'] = storage_var-required_to_make


            print('по окончанию цикла возвращает тот i который требуется', i)    
            data = graphic[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code)]['Дата_производства'].dt.date.iloc[i] 
            print('graphic2', graphic)
            return data

        
        ## иначе если возвращается только одна дата, то мы должны обнулить ее и присвоить дату
        graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code), 'Объем_фасовки, т'] = 0
        print('graphic3', graphic)
        return graphic[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code)]['Дата_производства'].dt.date.iloc[0] 

In [13]:
orders['дата производства'] = orders.apply(lambda x: data_shipment(x['Пункт отгрузки'], x['КССС мат'], x['осталось произвести, тн']), axis=1)

НАЧАЛО
Объем больше
graphic1:    Пункт отгрузки  Код КССС Дата_производства  Объем_фасовки, т
0            ВНП   1779443        2022-06-11              3.65
1            ВНП   1779443        2022-06-12              3.00
НАЧАЛО
Объем больше
graphic1:    Пункт отгрузки  Код КССС Дата_производства  Объем_фасовки, т
0            ВНП   1779443        2022-06-11             3.125
1            ВНП   1779443        2022-06-12             3.000
НАЧАЛО
Объем больше
graphic1:    Пункт отгрузки  Код КССС Дата_производства  Объем_фасовки, т
0            ВНП   1779443        2022-06-11             1.725
1            ВНП   1779443        2022-06-12             3.000
НАЧАЛО
объем меньше
в цикле
len_of_dates= 2
прибавили к накопительной следующий объем:  4.725
storage fresh,  4.725
required fresh,  2.1
обнуляем предыдущую дату
после обнуления:,    Пункт отгрузки  Код КССС Дата_производства  Объем_фасовки, т
0            ВНП   1779443        2022-06-11               0.0
1            ВНП   1779443       

In [14]:
graphic

,Пункт отгрузки,Код КССС,Дата_производства,"Объем_фасовки, т"
0,ВНП,1779443,2022-06-11,0.00
1,ВНП,1779443,2022-06-12,0.35


In [15]:
orders

,КССС мат,Пункт отгрузки,"осталось произвести, тн",дата производства
0,1779443,ВНП,0.350,2022-06-11
1,1779443,ВНП,0.525,2022-06-11
2,1779443,ВНП,1.400,2022-06-11
3,1779443,ВНП,2.100,2022-06-12
4,1779443,ВНП,0.350,2022-06-12
5,1779443,ВНП,1.400,2022-06-12
6,1779443,ВНП,0.175,2022-06-12
7,1779443,ВНП,0.350,2022-06-12
8,1779443,ВНП,0.700,-


In [16]:
#graphic.loc[(graphic['Пункт отгрузки']=='ПНОС') & (graphic['Код КССС']==207465), 'Объем_фасовки, т']

In [17]:
#graphic[(graphic['Пункт отгрузки']=="ПНОС") & (graphic['Код КССС']==20746)].empty

In [18]:
##делать колонку с тем, сколько не хватает еще бессмысленно, так как продукт может далее распределиться по другим заказам, 
#для которых хватает и тогда число не будет иметь значение

In [19]:
# проверить если не накопилась дважды или трижды